## Getting started

Let's start by importing the Ollama library. This library is modeled after the Ollama web API, making it easy to interact with Ollama through Python.

In [2]:
import ollama

## Make sure Ollama is running

Go to `http://localhost:11434` in your browser and make sure you get the response `Ollama is running`

## Pulling a model

We already pulled our model earlier, so this step is not necessary. It is included here for demonstration purposes.

In [5]:
# ollama.pull('tinyllama')

ProgressResponse(status='success', completed=None, total=None, digest=None)

## List available models

If you have other models installed, you can list them.

In [6]:
ollama.list()

ListResponse(models=[Model(model='tinyllama:latest', modified_at=datetime.datetime(2025, 3, 7, 17, 50, 22, 405498, tzinfo=TzInfo(-06:00)), digest='2644915ede352ea7bdfaff0bfac0be74c719d5d5202acb63a6fb095b52f394a4', size=637700138, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='1B', quantization_level='Q4_0')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 2, 7, 19, 53, 28, 828200, tzinfo=TzInfo(-06:00)), digest='0a8c266910232fd3291e71e5ba1e058cc5af9d411192cf88b6d30e92b6e73163', size=4683075271, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='7.6B', quantization_level='Q4_K_M')), Model(model='llama3.2:latest', modified_at=datetime.datetime(2025, 2, 7, 19, 42, 22, 49375, tzinfo=TzInfo(-06:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama',

Well 🤔 that's kind of ugly. The API response is being cast as a couple of nested objects. Let's just get the names of the models from the API response.

In [9]:
response = ollama.list()
model_names = [response_model.model for response_model in response.models]
print(model_names)

['tinyllama:latest', 'deepseek-r1:latest', 'llama3.2:latest']


## Start a chat

Chats can be performed either all at once or in "streaming" mode.